Google Drive上の音声ファイルを圧縮して容量を削減する。

---
# 使用方法

# 1. 設定
スクリプト先頭の３つの項目: FILE_PATH, BITRATE, KEEP_ORIGINAL を設定する。

### 1.1. FILE_PATH
この項目では、圧縮したい音声ファイルの場所を指定する。例えば以下。
```python
FILE_PATH = '共有/共有音源/A音源/A001-050/00-A001 01 ゆれる秋.mp3'
```
マイドライブ以下の完全なフォルダ構成を記述する必要がある点に注意。

音声ファイルではなくフォルダを指定した場合は、そのフォルダに含まれる
全てのファイルを圧縮する。例えば以下はA音源を全て圧縮する。
```python
FILE_PATH = '共有/共有音源/A音源'
```

### 1.2. BITRATE
圧縮後の音声ファイルのビットレート（音質）を指定する。
値が大きいほど音質がよい。通常、 192 [kbps] または 128 [kbps] を選択する。

192 [kbps] は、室内録音においては事実上の音質の限界であり、これより大きい値を指定する意味はない。叡風会のレコーダーで録音するとこのビットレートになる。

128 [kbps] は、多くの人にとって 192 [kbps] と遜色ない音質でありながら容量もそこそこ小さくバランスが良い。

尚、圧縮前の音声ファイルのビットレートが、ここで指定する `BITRATE` の値より既に小さかった場合、そのファイルはPASSされ、そのファイルに変更は加えられない。

### 1.3. KEEP_ORIGINAL
圧縮前のファイルを残すか/残さないか(True/False)を指定する。

True の場合、圧縮前のファイルは`orig.[元ファイルの名前].mp3`という名前に変更され、もとのディレクトリに残される。

False の場合、圧縮前のファイルはゴミ箱を含む全てのファイルシステムから完全に失われ、復元することはできない。

# 2. 実行
スクリプト左上の実行ボタンをクリックして実行する。
進捗状況はスクリプトの下に表示される。


In [ ]:
# スクリプト

# User Settings
FILE_PATH = '共有/共有音源/B音源/2022年(B160~)'
BITRATE = 128
KEEP_ORIGINAL = True

# env
MOUNT_POINT = '/content/drive' # is always '/content/drive' in Google Drive
BASE_PATH = f'{MOUNT_POINT}/MyDrive/{FILE_PATH}' # absolute path

# const
CODEC = 'mp3' # codec
(IGNORE, PASS, COMPRESS) = ('IGNORE', 'PASS', 'COMPRESS') # select-case

# install ffmpeg 
!pip install ffmpeg-python mutagen

# mount Google Drive on colab
from google.colab import drive
drive.mount(MOUNT_POINT)

# import libraries
import ffmpeg
import shutil
from os import path, remove, listdir as ls
import re
from mutagen.mp3 import MP3

# Search the given directory recursively and apply
# the cbf (callback-function) to all files
def tree(directory, callback): # pure
    for file in ls(directory):
        if path.isdir(f'{directory}/{file}'):
            tree(f'{directory}/{file}', callback)
        else:
            callback(f'{directory}/{file}')

# Select operations by file attributes.
def select_case(file_path):
    if re.match(r'.*\.mp3', file_path) is None:
        return IGNORE # file is not a mp3
    kbps = int(MP3(file_path).info.bitrate*1e-3)
    if (kbps <= BITRATE):
        print(f'{file_path}, {PASS}, {kbps}kbps')
        return PASS # original file size is small enough 
    else:
        print(f'{file_path}, {COMPRESS}, {kbps}kbps -> {BITRATE}kbps')
        return COMPRESS

# Wrap ffmpeg
def compress(file_path):
    orig = f'{path.dirname(file_path)}/orig.{path.basename(file_path)}'
    shutil.copyfile(file_path, orig)
    (ffmpeg
        .input(orig)
        .output(file_path, format=CODEC, audio_bitrate=f'{BITRATE}k')
        .run(overwrite_output=True, quiet=True))
    if not KEEP_ORIGINAL:
        remove(orig)

def execute(file_path):
    case = select_case(file_path)
    (case == IGNORE) and None
    (case == PASS) and None
    (case == COMPRESS) and compress(file_path)

### executable section
print('file name, action, bitrate')
if path.isdir(BASE_PATH):
    tree(directory=BASE_PATH, callback=execute)
else:
    execute(BASE_PATH)
print('done')